# Data Discovery in Data Lakes with BLEND

### Load libraries and define paths

In [4]:
import os
import sys
from pathlib import Path
import polars as pl
from tabulate import tabulate

In [6]:
data_path = Path("..", "data", "modena")

data_path.absolute(), data_path.exists()

(PosixPath('/home/nanni/projects/bdm/lab/data-discovery/exercises/../data/modena'),
 True)

Add BLEND modules to the PYTHONPATH list of paths

In [7]:
modules_path = Path("modules")
blend_module_path = modules_path.joinpath("BLEND")

sys.path.append(str(blend_module_path.resolve()))
sys.path

['/home/nanni/projects/general-data-science/correlation',
 '/home/nanni/projects/general-data-science/ULOD',
 '/home/nanni/projects/general-data-science/sloth',
 '/home/nanni/projects/general-data-science/JOSIE',
 '/home/nanni/projects/general-data-science/BLEND',
 '/home/nanni/mystuff/py_datafusion/src',
 '/home/nanni/projects/bdm/lab/data-discovery/exercises',
 '/home/nanni/miniconda3/lib/python313.zip',
 '/home/nanni/miniconda3/lib/python3.13',
 '/home/nanni/miniconda3/lib/python3.13/lib-dynload',
 '',
 '/home/nanni/projects/bdm/lab/data-discovery/.venv/lib/python3.13/site-packages',
 '/home/nanni/projects/bdm/lab/data-discovery/exercises/modules/BLEND']

In [8]:
db_path = data_path.joinpath("index_blend.db")
data_lake_path = data_path.joinpath("data-lake")
queries_path = data_path.joinpath("queries")

In [9]:
db_path.exists()

True

In [10]:
from blend import BLEND
from blend.utils import clean

### Instantiate BLEND index

In [11]:
index = BLEND(db_path)

### Load the query dataset

We have some datasets in the _query_ folder:

In [12]:
queries = sorted(os.listdir(queries_path))

print('\n\n'.join(queries))

Archi-stradali.csv

Risultati-di-lista-delle-elezioni-europee-del-2024.cpy-0.csv

ds115_economia_spesa_media_mese_categoria_area_residenza_2007-2013.csv

section_district_codes.csv


In [13]:
# select one of the available queries
query_table_idx = 1
query_table_name = queries[query_table_idx]

# load the query dataset
qdf = pl.read_csv(queries_path.joinpath(query_table_name))

qdf

THE_PK_KEY,SECTION,DISTRICT,DISTRICT_CD,DATA_TYPE,ELECTION,YEAR,FORZA_ITALIA,SUDTIROLER_VOLKS_PARTEI,STATI_UNITI_D_EUROPA,ALTERNATIVA_POPOLARE___PPE,PACE_TERRA_DIGNITA_,MOVIMENTO_5_STELLE,LIBERTA_,ALLEANZA_VERDI_SINISTRA,PARTITO_DEMOCRATICO,FRATELLI_D_ITALIA,SIAMO_EUROPEI___AZIONE_CALENDA,LEGA,#REGISTERED,#VOTERS,#DISPUTED,#BLANK,#VALID,FORM,#NOT VALID,#NULL
str,i64,str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64
"""1_Centro_Europee""",1,"""Centro""",1,"""Liste""","""Europee""",2024,30,null,28,8,11,20,null,48,136,186,null,23,881,533,0,6,525,"""E24""",8,2
"""2_Centro_Europee""",2,"""Centro""",1,"""Liste""","""Europee""",2024,29,3,11,1,13,null,null,33,null,95,22,null,732,444,0,3,431,"""E24""",13,10
null,5,"""Centro""",1,"""Liste""","""Europee""",2024,48,1,28,2,13,15,1,32,146,155,null,34,811,512,0,3,505,"""E24""",7,4
"""7_Centro_Europee""",7,"""Centro""",1,"""Liste""","""Europee""",2024,58,1,31,null,16,29,5,46,173,165,28,32,854,596,0,4,584,"""E24""",12,8
"""8_Centro_Europee""",8,"""Centro""",1,"""Liste""","""Europee""",2024,27,null,null,null,7,29,2,34,137,102,19,21,773,398,0,4,392,"""E24""",6,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,185,"""San Faustino""",null,"""Liste""","""Europee""",2024,21,1,12,0,8,null,1,56,266,153,32,21,845,618,0,5,609,"""E24""",9,4
"""187_Buon Pastore_Europee""",187,"""Buon Pastore""",3,"""Liste""","""Europee""",2024,11,0,9,1,3,31,1,35,204,93,13,19,663,426,0,3,420,"""E24""",6,3
"""188_Buon Pastore_Europee""",188,"""Buon Pastore""",3,"""Liste""","""Europee""",2024,21,null,19,2,18,34,4,36,239,82,18,29,718,521,0,5,502,"""E24""",19,14


## Keyword Search

In many use-cases, one of the simplest and most useful kind of data discovery task is the _keyword_ search.

Basically, we want to identify those datasets whose cell values, considered as a set, have the highest overlap with a user-given query set.

We don't check for any ordering on rows/columns, just the overlap.

**Q:** For which use-cases a keyword search is actually useful? Do we expect to have interesting results for this scenario?

In [14]:
# we flatten our query dataframe values to a set
values = list(set(map(clean, {cell for row in qdf.rows() for cell in row})))

len(values)

668

In [15]:
results = index.keyword_search(values, k=20)

print(f"Query table: {query_table_name}\n")

print(tabulate(results, headers=['dataset', 'overlap']))

Query table: Risultati-di-lista-delle-elezioni-europee-del-2024.cpy-0.csv

dataset                                                         overlap
------------------------------------------------------------  ---------
Risultati-di-lista-delle-elezioni-europee-del-2024.cpy-0            668
Risultati-di-lista-delle-elezioni-europee-del-2024                  667
Stradario-comunale                                                  537
Archi-stradali                                                      536
Pratiche-edilizie-anni-dal-2000-ad-oggi                             532
Pratiche-edilizie-anni-dal-1900-al-1999                             532
Sezioni-di-censimento                                               531
Servizio-di-scarico-risorsa-in-formato-CSV                          531
Risultati-delle-elezioni-europee-2019                               530
File-in-formato-csv                                                 519
Risultati-delle-elezioni-europee-2009                        

## Single Column JOIN Search

The dataset above has a single key column, _THE\_PK\_KEY_, which is the combination of _SECTION_, _DISTRICT\_CD_ and _ELECTION_

Such a combination might be useful to retrieve related tables using BLEND.

In [16]:
# extract and clean the values of the key
column = qdf.get_column('THE_PK_KEY').map_elements(lambda x: clean(x), pl.String).drop_nulls()

values = column.to_list()

column.head(5)

THE_PK_KEY
str
"""1_Centro_Europee"""
"""2_Centro_Europee"""
"""7_Centro_Europee"""
"""8_Centro_Europee"""
"""9_Centro_Europee"""


Execute the search with BLEND, returning the 10 columns with highest overlap with the query. If we run the query several times with the same input, we 
should see always the same results (ties may appear in different order).

In [17]:
results = index.single_column_join_search(values, k=20)

print(f"Query table: {query_table_name}\n")
print(tabulate(results, headers=['dataset', 'column idx', 'overlap (distinct)', 'overlap (general)']))

Query table: Risultati-di-lista-delle-elezioni-europee-del-2024.cpy-0.csv

dataset                                                               column idx    overlap (distinct)    overlap (general)
------------------------------------------------------------------  ------------  --------------------  -------------------
Affluenze-delle-elezioni-europee-2019                                          0                   128                  128
Risultati-di-lista-delle-elezioni-europee-del-2024                             0                   128                  128
Risultati-delle-elezioni-europee-2019                                          0                   128                  128
Affluenze-e-risultati-elettorali-delle-elezioni-europee-2024                   0                   128                  128
Risultati-di-lista-delle-elezioni-europee-del-2024.cpy-0                       0                   128                  128
Affluenze-delle-elezioni-europee-2009                    

We can now easily identify the datasets we are most interested with; we can load them by accessing the results list and check
their content.

In [18]:
r_df = pl.read_csv(data_lake_path.joinpath(f"{results[-1][0]}.csv"))
r_df

THE_PK_KEY,SECTION,DISTRICT,DISTRICT_CD,DATA_TYPE,ELECTION,YEAR,ELECTION_CD,#MALE_REGISTERED,#FEMALE_REGISTERED,#MALE_VOTERS,#FEMALE_VOTERS
str,i64,str,i64,str,str,i64,str,i64,i64,i64,i64
"""160_San Faustino_Europee""",160,"""San Faustino""",4,"""Affluenza""","""Europee""",2019,"""EUR_2019""",394,418,296,319
"""161_San Faustino_Europee""",161,"""San Faustino""",4,"""Affluenza""","""Europee""",2019,"""EUR_2019""",281,309,188,198
"""164_San Faustino_Europee""",164,"""San Faustino""",null,"""Affluenza""","""Europee""",2019,"""EUR_2019""",383,421,279,303
"""166_San Faustino_Europee""",166,"""San Faustino""",4,"""Affluenza""","""Europee""",2019,"""EUR_2019""",357,327,237,229
null,167,"""San Faustino""",4,"""Affluenza""","""Europee""",2019,null,395,434,296,325
…,…,…,…,…,…,…,…,…,…,…,…
"""154_San Faustino_Europee""",154,"""San Faustino""",4,"""Affluenza""","""Europee""",2019,"""EUR_2019""",340,383,249,280
"""156_San Faustino_Europee""",156,"""San Faustino""",4,"""Affluenza""","""Europee""",2019,null,373,394,262,273
"""157_San Faustino_Europee""",157,"""San Faustino""",4,"""Affluenza""","""Europee""",2019,"""EUR_2019""",359,374,256,253


## Multi-Column JOIN Search - Combination of keyword/single-JOIN searches

In many cases, a single column doesn't identify every record of a dataset, and a combination of different attributes is thus required.

Suppose that we do not have anymore the "THE_KEY" column.

In [ ]:
qdf = qdf.drop("THE_PK_KEY")
qdf.head()

Below, search joinable tables with the single-column approach on the specified columns, then aggregate the results and identify the tables that appear more often in the results.

**Q:** Each run produces the same result set?

**Q:** Do you find any shortcomings of this approach?

In [19]:
results_section =  ... 

In [20]:
results_district = ...

In [21]:
results_election = ...

In [ ]:
# there are many possible ways to aggregate results from above steps
# one is to use a Counter object or a defaultdict 
from collections import defaultdict, Counter

# you shold provide a sorted list of pairs <dataset_id, occurrences>
aggregation = ...

In [ ]:
print(tabulate(aggregation, ['dataset', 'occurrences']))

## Multi-Column JOIN Search - MATE algorithm

We can use a **multi-column search** approach. 

This is based on MATE (Multi-Attribute Table Extraction) algorithm, which allows us to search n-ary joins without any
other intermediate step.

In [ ]:
rows = qdf.select(['SECTION', 'DISTRICT', 'ELECTION'])

# here we simply clean the cell values,
# to put them in the same format used
# in the index
values = [
    list(
        map(
            lambda x: clean(x), 
            row
        )
    ) for row in rows.rows()
]

print(tabulate(values[:10]))

In [ ]:
mc_results = index.multi_column_join_search(values, 10, verbose=True)

In [ ]:
print(f"Query table: {query_table_name}\n")
print(tabulate(mc_results, headers=['dataset', 'columns', 'join_score']))

The order of the columns **doesn't affect** the final results, but might impact the efficiency (see section 6.1 of MATE paper if you are interested).

The final order in the top-K might slightly change, but overall the top-K tables are the same.

We can swap the columns used before:

In [ ]:
rows = qdf.select(['DISTRICT', 'SECTION', 'ELECTION'])

values = [list(map(lambda x: clean(x), row)) for row in rows.rows()]

print(tabulate(values[:5]))

In [ ]:
mc_results_v2 = index.multi_column_join_search(values, 10, verbose=True)

In [ ]:
# the results are the same obtained above
print(tabulate(mc_results_v2, headers=['dataset', 'columns', 'join_score']))

In [ ]:
tables_from_run_1 = {r[0] for r in mc_results}
tables_from_run_2 = {r[0] for r in mc_results_v2}

len(tables_from_run_1.intersection(tables_from_run_2)), tables_from_run_1.difference(tables_from_run_2), tables_from_run_2.difference(tables_from_run_1)

We can do another test with a different combination of the same three columns:

In [ ]:
qdf.get_column('ELECTION').unique()

In [ ]:
rows = qdf.select(['ELECTION', 'DISTRICT', 'SECTION'])
values = [list(map(lambda x: clean(x), row)) for row in rows.rows()]
print(tabulate(values[:3]))

In [ ]:
mc_results_v3 = index.multi_column_join_search(values, 10, verbose=False)

In [ ]:
tables_from_run_1 = {r[0] for r in mc_results}
tables_from_run_2 = {r[0] for r in mc_results_v2}
tables_from_run_3 = {r[0] for r in mc_results_v3}

len(tables_from_run_1.intersection(tables_from_run_2).intersection(tables_from_run_3))

Another "desiderata" when working with multi column join discovery, is the possibility to easily add/discard columns

For instance, we can add the "YEAR" column to our query:

In [ ]:
rows = qdf.select(['SECTION', 'DISTRICT', 'YEAR', 'ELECTION']).rows()

values = [list(map(lambda x: clean(x), row)) for row in rows]

results = index.multi_column_join_search(values, 10)

print(tabulate(results, ['dataset', 'columns', 'join_score']))

In [ ]:
rows = qdf.select(['SECTION', 'DISTRICT', 'YEAR', 'ELECTION', 'DATA_TYPE']).rows()

values = [list(map(lambda x: clean(x), row)) for row in rows]

results = index.multi_column_join_search(values, 10)

print(tabulate(results, ['dataset', 'columns', 'join_score']))

# Example: Parkings and Shops

A user needs to check if shops in Modena have enough parking for people with disabilities: take the "Archi stradali" dataset, and from there identify datasets about parkings for people with disabilities and shops and join them to create a unified view of these data.

In [ ]:
query_table_idx = 0
query_table_name = queries[query_table_idx]
print(f"Query dataset: {query_table_name}")

qdf = pl.read_csv(queries_path.joinpath(query_table_name))

qdf.head()

In [ ]:
# on which column(s) do we need to work?
query_values = ...

In [ ]:
# which kind of query do we need to perform? 
results = ... # index.? 

In [ ]:
# Are there any relevant dataset(s) in our results list? What we can we do in the end to obtain a final and valid view?

# Example: Correlated Expense

We have a query dataset about the average monthly familiar expense in a wide list of categories.

You have to identify datasets that join on its key attributes and that are also correlated with the expense information.

- Do any dataset appear more than once, with different key attribute for the join-correlation search?

- Is there any significant shift in the mean on the after-join dataset?

In [ ]:
query_table_idx = 2
query_table_name = queries[query_table_idx]
print(f"Query dataset: {query_table_name}")

qdf = pl.read_csv(os.path.join(queries_path, query_table_name))

qdf

In [ ]:
# use different key columns to generate different results
join_key = 'YEAR'
# join_key = 'CATEGORY'

# what results do you find with the given join key?
correlation_search_results = ...

In [ ]:
# from the datasets identified above, has the mean of the numerical columns changed, from the
# original to the after-join dataset? Why?
# 
# If so, keeping in mind how the QCR scheme works, how do you think that this affects the search?